## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt

## 데이터 크기 정의하기

In [ ]:
NoOfData    = 180  # 정상/고장 데이터 각 180개씩 
NoOfSensor  = 3    # 가속도(Acceleration), 전압(Voltage), 전류(Current)
NoOfFeature = 10   # 특징 개수:10개 (순서: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Shape factor, Impulse factor)

NoOfData, NoOfSensor, NoOfFeature

## Time Domain 특징값 추출 (10 features * 3 sensors = 30개씩)

In [ ]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [ ]:
# 특징데이터 크기 지정
TimeFeature_Normal   = np.zeros((NoOfData, NoOfSensor*NoOfFeature))
TimeFeature_Abnormal = np.zeros((NoOfData, NoOfSensor*NoOfFeature))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d.csv'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d.csv'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',')         # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',')         # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[i, 10*j+0] = np.max(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+1] = np.min(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+2] = np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+3] = rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+4] = np.var(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+5] = sp.skew(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+6] = sp.kurtosis(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+7] = np.max(temp_data1.iloc[:,j])/rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[i, 10*j+8] = rms(temp_data1.iloc[:,j])/np.mean(np.abs(temp_data1.iloc[:,j]))
        TimeFeature_Normal[i, 10*j+9] = np.max(temp_data1.iloc[:,j])/np.mean(np.abs(temp_data1.iloc[:,j]))
        
        # Abnormal Time Domain Feature
        TimeFeature_Abnormal[i, 10*j+0] = np.max(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+1] = np.min(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+2] = np.mean(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+3] = rms(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+4] = np.var(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+5] = sp.skew(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+6] = sp.kurtosis(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+7] = np.max(temp_data2.iloc[:,j])/rms(temp_data2.iloc[:,j])
        TimeFeature_Abnormal[i, 10*j+8] = rms(temp_data2.iloc[:,j])/np.mean(np.abs(temp_data2.iloc[:,j]))
        TimeFeature_Abnormal[i, 10*j+9] = np.max(temp_data2.iloc[:,j])/np.mean(np.abs(temp_data2.iloc[:,j]))
        
print(TimeFeature_Normal.shape)
print(TimeFeature_Abnormal.shape)

Normal, Abnormal 데이터 합치기  

In [ ]:
TimeFeature = np.concatenate([TimeFeature_Normal, TimeFeature_Abnormal] , axis=0)
TimeFeature.shape

.

.

.



## Frequency Domain 특징값 추출 (10 features \* 8 wavelet levels * 3 sensors = 240개씩)

In [ ]:
# Wavelet options

MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 8                            # Wavelet 분해 레벨 지정
select  = 8                            # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [ ]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfData, NoOfSensor*NoOfFeature*select))
FreqFeature_Abnormal = np.zeros(shape=(NoOfData, NoOfSensor*NoOfFeature*select))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d.csv'%(i+1)    # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d.csv'%(i+1)  # Abnormal 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1 , sep=',')) # 임시 Normal 데이터
    temp_data2 = np.array(pd.read_csv(temp_path2 , sep=',')) # 임시 Abnormal 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_data2, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in range(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
            
            # Normal Frequency Domain Feature
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+0] = np.max(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+1] = np.min(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+2] = np.mean(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+3] = rms(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+4] = np.var(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+5] = sp.skew(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+6] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+7] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+8] = rms(coef1[:,j])/np.mean(np.abs(coef1[:,j]))
            FreqFeature_Normal[i, NoOfFeature*j*select+k*NoOfFeature+9] = np.max(coef1[:,j])/np.mean(np.abs(coef1[:,j]))
            
            # Abnormal Frequency Domain Feature
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+0] = np.max(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+1] = np.min(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+2] = np.mean(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+3] = rms(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+4] = np.var(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+5] = sp.skew(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+6] = sp.kurtosis(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+7] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+8] = rms(coef2[:,j])/np.mean(np.abs(coef2[:,j]))
            FreqFeature_Abnormal[i, NoOfFeature*j*select+k*NoOfFeature+9] = np.max(coef2[:,j])/np.mean(np.abs(coef2[:,j]))

print(FreqFeature_Normal.shape)
print(FreqFeature_Abnormal.shape)

Normal, Abnormal 특징값 합치기      

In [ ]:
FreqFeature = np.concatenate([FreqFeature_Normal, FreqFeature_Abnormal] , axis=0)
FreqFeature.shape

.

.

.



## Feature 통합 (Time domain 30개 + Freq. domain 240 = 270개)

In [ ]:
Features = np.concatenate([TimeFeature, FreqFeature] , axis=1)

print("Feature Data Size :", Features.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 특징값으로 구성됨"%(Features.shape[0], Features.shape[0]/2, Features.shape[1]))

저장에 용이한 Data frame 형식으로 변환

In [ ]:
FeatureData = pd.DataFrame(Features)
FeatureData

추출된 특징 데이터 저장 (.csv 파일)

In [ ]:
path = './ProcessedData/FeatureData.csv'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)